In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import sympy as sp  # used for data storage and debugging purposes: 
import numpy as np
import random

In [2]:
# This class creates a markov chain

class MarkovChain:
    def __init__(self, t_matrix, stages, states, noise):
        self.t_matrix = t_matrix
        self.stages = list(range(1,stages+1))
        self.states = list(range(1, states +1))
        self.noise  = noise

    def nodes(self):
        nodes = [(1,1)]
        for i in self.stages[1:]:
            for j in self.states:
                nodes.append((i,j))
        return nodes
    
    def edges(self):
        edges = {}
        for j in self.states:
                edges[(1,1),(2,j)] = t_matrix[1][j-1]
        for i in self.stages[2:]:
            for j in self.states:
                for k in self.states:
                    edges[(i-1,j),(i,k)] = t_matrix[i-1][j-1][k-1]
        return edges


    def sample_path(self):
        all_edges = self.edges()
        path = [(1,1)]
        for i in self.stages[1:]: 
            path.append((i,random.choice(self.states)))
        return path
    
    def sample_noise(self):
        noise = []
        for i in self.stages:
            noise.append(random.choice(list(self.noise.keys())))
        return noise

# This class is meant to give every node children and parent attribute useful later for the algorithm.
class Node:
    def __init__(self, node, stages, states):
        self.node_name = node
        self.states = list(range(1, states +1))
        
        if node[0] == 1:
            self.parents  = None
            self.children = [(2,s) for s in self.states]
        elif node[0] == 2:
            self.parents = [(1,1)]
            self.children = [(3,s) for s in self.states]
        elif node[0] > 2 &  node[0] < stages:
            self.parents = [(node[0]-1,s) for s in self.states]
            self.children = [(node[0]+1,s) for s in self.states]
        elif node[0] == stages:
            self.parents  = [(node[0]-1,s) for s in self.states]
            self.children = None


# This class is meant to define the nodes on every stage, useful later for the algorithm. 
class Stage:
    def __init__(self, stages, states):
        self.stages = list(range(1,stages+1))
        self.states = list(range(1, states +1))
        self.nodes = {}
        
        for i in self.stages:
            if i == 1:
                self.nodes[i] = [(1,1)]
            else:
                self.nodes[i] = [(i,j) for j in self.states]


In [3]:
intra = {1: {"demand": {1: [100,130,100], 2: [200,140,200], 3: [100,210,150]}, 'rho':0.3},
         2: {"demand": {1: [200,230,150], 2: [300,330,200], 3: [200,100,300]}, 'rho':0.4},
         3: {"demand": {1: [300,400,200], 2: [400,100,200], 3: [300,500,400]}, 'rho':0.3}}

OmegaRow = {1,2,3}
OmegaBus = {1,2,3}
OmegaT   = {1,2,3}

line_cost = {1: 10, 2: 10, 3: 10}
B_ij = {1: 0.40, 2: 0.40, 3: 0.40}
S_ij = {1: 100, 2: 100, 3: 100}
n0 = {1: 1, 2: 1, 3: 1}

p_max = {1: 100, 2: 100, 3: 100} 

branch = {1: (1,2), 2: (1,3), 3: (2,3)}

t_matrix= [1,
 [0.3, 0.4, 0.3],
 [[0.3, 0.4, 0.3], [0.3, 0.4, 0.3], [0.3, 0.4, 0.3]]]

stages = 3
states = 3


init_state = {"i_coal_x": {1:0, 2:0, 3:0},
              "x_line_x": {1:0, 2:0, 3:0}}



In [4]:
# Creating the tree structure using the inputs
mc = MarkovChain(t_matrix, stages, states, intra)  

# Creating a dictionary containing node info such as children and parents 
Nodes = {}
for i in mc.nodes():                 
    Nodes[i] = Node(i,stages,states)

#Defining the probabilities of the edges for cut calculation 
Pij = mc.edges()

#A dictionary containing Stages in each node
Stages = Stage(stages,states)

In [5]:
def subproblem_builder():
    nodes = mc.nodes()
    subproblems = {}
    for i in nodes:
            stage = i[0]
            state = i[1]
            print("subproblem", i, "stage", stage, "state", state)

            m = ConcreteModel()
            
            # Identify the state variables 
        
            m.i_coal =    Var(OmegaBus, within = NonNegativeReals)
            m.i_coal_in = Var(OmegaBus, within = NonNegativeReals) 
            m.x_line =    Var(OmegaRow, within = Binary)
            m.x_line_in = Var(OmegaRow, within = Binary)

            m.i_coal.state_variable = True
            m.x_line.state_variable = True


            m.i_coal_loc =  Var(OmegaBus, within = NonNegativeReals)
            m.x_line_loc =  Var(OmegaRow, within = Binary)


            # Identify the dummy constraint parameters:
            
            m.i_coal_x = Param(OmegaBus, initialize=0, mutable=True)
            m.x_line_x = Param(OmegaRow, initialize=0, mutable=True)

            m.i_coal_x.dummy = True
            m.x_line_x.dummy = True
            
            #Local Variables
            
            m.cost_to_go = Var(within=NonNegativeReals, initialize=0)
            m.p_coal  = Var(OmegaBus, OmegaT, within=NonNegativeReals, initialize=0)
            m.p_exist = Var(OmegaBus, OmegaT, within=NonNegativeReals, initialize=0)
            m.p_shed  = Var(OmegaBus, OmegaT,  within=NonNegativeReals, initialize=0)
            m.flow    = Var(OmegaRow, OmegaT,  within=NonNegativeReals, initialize=0)
            m.theta   = Var(OmegaBus, OmegaT,  within=NonNegativeReals, initialize=0)

            #Identify Noise Parameters

            m.demand    = Param(OmegaBus, OmegaT,  initialize= 50, mutable=True)
            m.demand.noise = True

            #Suffix for backward pass
            m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)


            m.obj  = Objective(expr = sum(50*m.i_coal_loc[n] for n in OmegaBus) + sum(line_cost[l]*m.x_line_loc[l] for l in OmegaRow)
                                    + sum(5*m.p_exist[n,t] + 2*m.p_coal[n,t] + 10*m.p_shed[n,t] for n in OmegaBus for t in OmegaT) + m.cost_to_go, sense= minimize)


            def nodbal(m,n,t):
                return sum(m.flow[l,t] for l in OmegaRow if branch[l][1] == n) - sum(m.flow[l,t] for l in OmegaRow if branch[l][0] == n)\
                           + m.p_coal[n,t] + m.p_exist[n,t] + m.p_shed[n,t] - m.demand[n,t] == 0 
            m.nodbal = Constraint(OmegaBus, OmegaT,  rule=nodbal)

                
            def pflow(m,l,t):
                return  m.flow[l,t] == (1/B_ij[l])*(m.theta[branch[l][0],t] - m.theta[branch[l][1],t])
            m.pflow = Constraint(OmegaRow, OmegaT,  rule= pflow)
    
            
            def fcap1(m,l,t):
                return m.flow[l,t] - S_ij[l]*(n0[l] + m.x_line_in[l]) <= 0
            m.fcap1 = Constraint(OmegaRow,  OmegaT,  rule = fcap1)
            
            def fcap2(m,l,t):
                return - m.flow[l,t] - S_ij[l]*(n0[l] + m.x_line_in[l]) <= 0
            m.fcap2 = Constraint(OmegaRow,  OmegaT, rule = fcap2)
    
            def maxp1(m,n,t):
                 return  m.p_exist[n,t] <= p_max[n] 
            m.maxp1 = Constraint(OmegaBus,  OmegaT,  rule = maxp1)

            def maxc1(m,n,t):
                 return m.p_coal[n,t]  <= 100*m.i_coal_in[n]
            m.maxc1 = Constraint(OmegaBus,  OmegaT,  rule = maxc1)


            def pshed(m,n,t):
                 return m.p_shed[n,t] <= m.demand[n,t]
            m.pshed = Constraint(OmegaBus,  OmegaT,  rule = pshed)

            def ramp1(m,n,t):
                 return m.p_coal[n,t] - m.p_coal[n,t-1] <= 20
            m.ramp1 = Constraint(OmegaBus, OmegaT-{1}, rule = ramp1)

            def ramp2(m,n,t):
                 return m.p_coal[n,t] - m.p_coal[n,t-1] >= -20
            m.ramp2 = Constraint(OmegaBus, OmegaT-{1}, rule = ramp2)

            def ramp3(m,n,t):
                 return m.p_exist[n,t] - m.p_exist[n,t-1] <= 20
            m.ramp3 = Constraint(OmegaBus, OmegaT-{1}, rule = ramp3)

            def ramp4(m,n,t):
                 return m.p_exist[n,t] - m.p_exist[n,t-1] >= -20
            m.ramp4 = Constraint(OmegaBus, OmegaT-{1}, rule = ramp4)


            def con_i_coalx(m,n):
                return m.i_coal_in[n] == m.i_coal_x[n]
            m.con_i_coalx = Constraint(OmegaBus, rule = con_i_coalx)


            def con_x_linex(m,l):
                return m.x_line_in[l] == m.x_line_x[l]
            m.con_x_linex = Constraint(OmegaRow, rule = con_x_linex)


            def con_i_coalada(m,n):
                return m.i_coal[n] == m.i_coal_in[n] + m.i_coal_loc[n]
            m.con_i_coalada = Constraint(OmegaBus, rule = con_i_coalada)


            def con_x_lineada(m,l):
                return m.x_line[l] == m.x_line_in[l] + m.x_line_loc[l]
            m.con_x_lineada = Constraint(OmegaRow, rule = con_x_lineada)
        
            m.cuts = ConstraintList()

            #define dummy_constraints
            
            m.con_i_coalx.dummy_constraint = True
            m.con_x_linex.dummy_constraint = True

            subproblems[i] = m
    return subproblems

sb = subproblem_builder()
opt = SolverFactory('gurobi')


subproblem (1, 1) stage 1 state 1
subproblem (2, 1) stage 2 state 1
subproblem (2, 2) stage 2 state 2
subproblem (2, 3) stage 2 state 3
subproblem (3, 1) stage 3 state 1
subproblem (3, 2) stage 3 state 2
subproblem (3, 3) stage 3 state 3


In [6]:
iteration_noise = [3,1,1]
for v in sb[2,1].component_objects(Param,active=True):    
    if hasattr(v, 'noise') == True:
        for index in v: 
            if index == None:
                setattr(sb[1,1], str(v), intra[iteration_noise[0]][str(v.name)][index[0]][index[1]-1]) 
            else:
                v[index].value = intra[iteration_noise[0]][str(v.name)][index[0]][index[1]-1]

sb[2,1].demand.display()

demand : Size=9, Index=demand_index, Domain=Any, Default=None, Mutable=True
    Key    : Value
    (1, 1) :   300
    (1, 2) :   400
    (1, 3) :   200
    (2, 1) :   400
    (2, 2) :   100
    (2, 3) :   200
    (3, 1) :   300
    (3, 2) :   500
    (3, 3) :   400


In [7]:
fpi = {}
bpi = {}  #Currently Unused but should be used

# The following code should be turned into functions. namely, the forward and backward passes. r

for iter in range(30):

    print("Forward pass", iter)
    
    iteration_path = mc.sample_path()
    iteration_noise = mc.sample_noise()

    print("Iteration_noise", iteration_noise)
    for step, node in enumerate(iteration_path):
    
        print("\nnode", node)
    
        if node == (1, 1):

            sb[node].x_line.domain = Binary        # This is because in the backward pass we relax these into continuous variables
            sb[node].x_line_in.domain = Binary
            sb[node].x_line_loc.domain = Binary


            for v in sb[1,1].component_objects(Param,active=True):
                if hasattr(v, 'dummy') == True:
                    for index in v: 
                        if index == None:
                            setattr(sb[1,1], str(v), init_state[str(v)]) 
                        else:
                            setattr(sb[1,1], str(v[index]), init_state[str(v)][index])

            for v in sb[1,1].component_objects(Param,active=True):    
                if hasattr(v, 'noise') == True:
                    for index in v: 
                        if index == None:
                            setattr(sb[1,1], str(v), intra[iteration_noise[0]][str(v.name)][index[0]][index[1]-1]) 
                        else:
                            v[index].value = intra[iteration_noise[0]][str(v.name)][index[0]][index[1]-1]

            # for n in OmegaBus:
            #     for t in OmegaT:
            #         sb[1,1].demand[n,t].value = intra[iteration_noise[0]]["demand"][n][t-1]


        
        else:
            sb[node].x_line.domain = Binary        # This is because in the backward pass we relax these into continuous variables
            sb[node].x_line_in.domain = Binary
            sb[node].x_line_loc.domain = Binary


            for v in sb[node].component_objects(Param,active=True):
                if hasattr(v, 'dummy') == True:
                    for index in v: 
                        if index == None:
                            setattr(sb[node], str(v), fpi[iter,step-1]['var_out'][(0,str(v)[:-2])]) 
                        else:
                            setattr(sb[node], str(v[index]), fpi[iter,step-1]['var_out'][(index, str(v)[:-2])]) 

            for v in sb[node].component_objects(Param,active=True):
                if hasattr(v, 'noise') == True:
                    for index in v: 
                        if index == None:
                            setattr(sb[node], str(v), intra[iteration_noise[step]][str(v.name)][index[0]][index[1]-1]) 
                        else:
                            v[index].value = intra[iteration_noise[step]][str(v.name)][index[0]][index[1]-1]

            # for n in OmegaBus:
            #     for t in OmegaT:
            #         sb[node].demand[n,t].value = intra[iteration_noise[step]]["demand"][n][t-1]
                            
        results = opt.solve(sb[node])
    
        out_dict = {}
        
        for v in sb[node].component_objects(Var,active=True):
             if hasattr(v, 'state_variable') == True:
                for index in v: 
                    if index == None:
                        out_dict[0, v.name] = value(v[index])
                    else:
                        out_dict[index, v.name] = value(v[index])
    
        fpi[iter,step] = {'node'  : node,
                     'var_out': out_dict,
                     'obj_val': value(sb[node].obj),
                     'cos2go': sb[node].cost_to_go.value,
                     'stg_obj':  value(sb[node].obj) - sb[node].cost_to_go.value,
                     'noise': "None"
                     }
    
        print("Cost to Go (Bellman Term):", fpi[iter,step]['cos2go'])
        print("Objective Value:",fpi[iter,step]['obj_val'])
        print("Stage Objective:", fpi[iter,step]['stg_obj'])


#### -----------------------------##### -------------------------------- ##### 

    print("\nbackward pass")
    
    β = {}
    α = {}
    
    V = {}
    dxdV = {}
    
    # -----------------stage 4 cuts -------------#

    for stage in reversed(Stages.stages):
        print("current stage", stage)

        if stage > 1 :
            
            for node in Stages.nodes[stage]:
                print("Current node", node)

                for v in sb[node].component_objects(Param,active=True):
                     if hasattr(v, 'dummy') == True:
                        for index in v: 
                            if index == None:
                                v.value = fpi[iter,stage-2]['var_out'][(0,str(v.name)[:-2])]
                                print("input of {} set to {}".format(v, v.value))
                            else:
                                v[index].value = fpi[iter,stage-2]['var_out'][(index, str(v.name)[:-2])]
                                print("input of {} is {}".format(v[index], v[index].value))




                if stage == Stages.stages[-1]:
                    sb[node].cost_to_go.fix(0)

                sb[node].x_line.domain = NonNegativeReals         ### Just a temporary form of relaxation because the problem needs to be linear to find the duals variables. 
                sb[node].x_line_in.domain = NonNegativeReals
                sb[node].x_line_loc.domain = NonNegativeReals

                for o in intra:
                    for v in sb[node].component_objects(Param,active=True):
                        if hasattr(v, 'noise') == True:
                            for index in v: 
                                if index == None:
                                    setattr(sb[node], str(v), intra[o][str(v.name)][index[0]][index[1]-1]) 
                                else:
                                    v[index].value = intra[o][str(v.name)][index[0]][index[1]-1]
                                    
                    # for n in OmegaBus:
                    #     for t in OmegaT:
                    #         sb[node].demand[n,t].value = intra[o]["demand"][n][t-1]
                                                
                    results = opt.solve(sb[node])
                    V[node,o] = value(sb[node].obj)

                    for v in sb[node].component_objects(Constraint,active=True):
                        if hasattr(v, 'dummy_constraint') == True:
                            for index in v: 
                                    dxdV[node,str(v)[4:-1],index,o] = sb[node].dual[v[index]]
                                    print("dual for ", v, "is", sb[node].dual[v[index]])
                                    
                for i in Nodes[node].parents:

                    for v in sb[node].component_objects(Var,active=True):
                        if hasattr(v, 'state_variable') == True:
                            for index in v:
                                β[iter, i, node, str(v), index]  = Pij[i,node]*sum(intra[o]["rho"]*dxdV[node, str(v), index, o] for o in intra)


                    α[iter, i,node] = Pij[i,node]*sum(intra[o]["rho"]*V[node,o] for o in intra)\
                                 - sum(β[iter, i,node,v.name[:-2],index]*v[index].value\
                                           for v in sb[node].component_objects(Param,active=True)\
                                           if hasattr(v, 'dummy') == True\
                                           for index in v)


    
            for i in Stages.nodes[stage-1]:
                sb[i].cuts.add(expr = sb[i].cost_to_go >= sum(α[iter, i,j] for j in Stages.nodes[stage])\
                                       + sum(sum(β[iter, i,j,v.name,index]*v[index] for j in Stages.nodes[stage])\
                                           for v in sb[i].component_objects(Var,active=True)\
                                           if hasattr(v, 'state_variable') == True for index in v))
        
    for o in intra:
        for v in sb[1,1].component_objects(Param,active=True):
            if hasattr(v, 'noise') == True:
                for index in v: 
                    if index == None:
                        setattr(sb[1,1], str(v), intra[o][str(v.name)][index[0]][index[1]-1]) 
                    else:
                        v[index].value = intra[o][str(v.name)][index[0]][index[1]-1]


        # for n in OmegaBus:
        #     for t in OmegaT:
        #         sb[1,1].demand[n,t].value = intra[o]["demand"][n][t-1]

        results = opt.solve(sb[1,1])
        V[(1,1),o] = value(sb[1,1].obj)
    lower_bound = sum(intra[o]["rho"]*V[(1,1),o] for o in intra)
    print(lower_bound)

Forward pass 0
Iteration_noise [3, 2, 1]

node (1, 1)


Cost to Go (Bellman Term): 0.0
Objective Value: 23500.0
Stage Objective: 23500.0

node (2, 2)
Cost to Go (Bellman Term): 0.0
Objective Value: 15600.0
Stage Objective: 15600.0

node (3, 3)
Cost to Go (Bellman Term): 0.0
Objective Value: 8800.0
Stage Objective: 8800.0

backward pass
current stage 3
Current node (3, 1)
input of i_coal_x[1] is 0.0
input of i_coal_x[2] is 0.0
input of i_coal_x[3] is 0.0
input of x_line_x[1] is 0.0
input of x_line_x[2] is 0.0
input of x_line_x[3] is 0.0
dual for  con_i_coalx is -2150.0
dual for  con_i_coalx is -2400.0
dual for  con_i_coalx is -1900.0
dual for  con_x_linex is 0.0
dual for  con_x_linex is 0.0
dual for  con_x_linex is 0.0
dual for  con_i_coalx is -2400.0
dual for  con_i_coalx is -2400.0
dual for  con_i_coalx is -1900.0
dual for  con_x_linex is 0.0
dual for  con_x_linex is 0.0
dual for  con_x_linex is 0.0
dual for  con_i_coalx is -2400.0
dual for  con_i_coalx is -2400.0
dual for  con_i_coalx is -2400.0
dual for  con_x_linex is 0.0
dual for  con_

In [12]:
fpi = {}

# m.con_i_coalx.dummy_constraint = True
# m.con_x_linex.dummy_constraint = True
# m.i_coal.state_variable = True
# m.x_line.state_variable = True
# m.i_coal_x.dummy = True
# m.x_line_x.dummy = True
# m.demand.noise = True


iteration_path = mc.sample_path()
iteration_noise = mc.sample_noise()


node = (1,1)


sb[node].i_coal_x[1] = 0
sb[node].i_coal_x[2] = 0
sb[node].i_coal_x[3] = 0

sb[node].x_line_x[1] = 0
sb[node].x_line_x[2] = 0
sb[node].x_line_x[3] = 0

for n in OmegaBus:
    sb[node].demand[n].value = intra[iteration_noise[0]]["demand"][n]

results = opt.solve(sb[node])


out_dict = {}
        
out_dict[1, "i_coal"] = sb[node].i_coal[1].value
out_dict[2, "i_coal"] = sb[node].i_coal[2].value
out_dict[3, "i_coal"] = sb[node].i_coal[3].value

out_dict[1, "x_line"] = sb[node].x_line[1].value
out_dict[2, "x_line"] = sb[node].x_line[2].value
out_dict[3, "x_line"] = sb[node].x_line[3].value

fpi[1,0] =      {'node'  : node,
                'var_out': out_dict,
                'obj_val': value(sb[node].obj),
                'cos2go': sb[node].cost_to_go.value,
                'stg_obj':  value(sb[node].obj) - sb[node].cost_to_go.value,
                'noise': (iteration_noise[0], intra[iteration_noise[0]]["demand"][n])
                }

fpi

{(1, 0): {'node': (1, 1),
  'var_out': {(1, 'i_coal'): 0.0,
   (2, 'i_coal'): 0.0,
   (3, 'i_coal'): 0.0,
   (1, 'x_line'): 0.0,
   (2, 'x_line'): 0.0,
   (3, 'x_line'): 0.0},
  'obj_val': 14500.0,
  'cos2go': 0.0,
  'stg_obj': 14500.0,
  'noise': (2, 400)}}

In [13]:
node = iteration_path[1]


sb[node].i_coal_x[1] = fpi[1,0]['var_out'][1,'i_coal']
sb[node].i_coal_x[2] = fpi[1,0]['var_out'][2,'i_coal']
sb[node].i_coal_x[3] = fpi[1,0]['var_out'][3,'i_coal']

sb[node].x_line_x[1] = fpi[1,0]['var_out'][1,'x_line']
sb[node].x_line_x[2] = fpi[1,0]['var_out'][2,'x_line']
sb[node].x_line_x[3] = fpi[1,0]['var_out'][3,'x_line']

for n in OmegaBus:
    sb[node].demand[n].value = intra[iteration_noise[1]]["demand"][n]

results = opt.solve(sb[node])


out_dict = {}
        
out_dict[1, "i_coal"] = sb[node].i_coal[1].value
out_dict[2, "i_coal"] = sb[node].i_coal[2].value
out_dict[3, "i_coal"] = sb[node].i_coal[3].value

out_dict[1, "x_line"] = sb[node].x_line[1].value
out_dict[2, "x_line"] = sb[node].x_line[2].value
out_dict[3, "x_line"] = sb[node].x_line[3].value

fpi[1,1] =      {'node'  : node,
                'var_out': out_dict,
                'obj_val': value(sb[node].obj),
                'cos2go': sb[node].cost_to_go.value,
                'stg_obj':  value(sb[node].obj) - sb[node].cost_to_go.value,
                'noise': (iteration_noise[1], intra[iteration_noise[1]]["demand"][n])
                }

fpi

{(1, 0): {'node': (1, 1),
  'var_out': {(1, 'i_coal'): 0.0,
   (2, 'i_coal'): 0.0,
   (3, 'i_coal'): 0.0,
   (1, 'x_line'): 0.0,
   (2, 'x_line'): 0.0,
   (3, 'x_line'): 0.0},
  'obj_val': 14500.0,
  'cos2go': 0.0,
  'stg_obj': 14500.0,
  'noise': (2, 400)},
 (1, 1): {'node': (2, 2),
  'var_out': {(1, 'i_coal'): 0.0,
   (2, 'i_coal'): 0.0,
   (3, 'i_coal'): 0.0,
   (1, 'x_line'): 0.0,
   (2, 'x_line'): 0.0,
   (3, 'x_line'): 0.0},
  'obj_val': 13500.0,
  'cos2go': 0.0,
  'stg_obj': 13500.0,
  'noise': (3, 500)}}

In [14]:
node = iteration_path[2]


sb[node].i_coal_x[1] = fpi[1,1]['var_out'][1,'i_coal']
sb[node].i_coal_x[2] = fpi[1,1]['var_out'][2,'i_coal']
sb[node].i_coal_x[3] = fpi[1,1]['var_out'][3,'i_coal']

sb[node].x_line_x[1] = fpi[1,1]['var_out'][1,'x_line']
sb[node].x_line_x[2] = fpi[1,1]['var_out'][2,'x_line']
sb[node].x_line_x[3] = fpi[1,1]['var_out'][3,'x_line']

for n in OmegaBus:
    sb[node].demand[n].value = intra[iteration_noise[1]]["demand"][n]

results = opt.solve(sb[node])


out_dict = {}
        
out_dict[1, "i_coal"] = sb[node].i_coal[1].value
out_dict[2, "i_coal"] = sb[node].i_coal[2].value
out_dict[3, "i_coal"] = sb[node].i_coal[3].value

out_dict[1, "x_line"] = sb[node].x_line[1].value
out_dict[2, "x_line"] = sb[node].x_line[2].value
out_dict[3, "x_line"] = sb[node].x_line[3].value

fpi[1,1] =      {'node'  : node,
                'var_out': out_dict,
                'obj_val': value(sb[node].obj),
                'cos2go': sb[node].cost_to_go.value,
                'stg_obj':  value(sb[node].obj) - sb[node].cost_to_go.value,
                'noise': (iteration_noise[2], intra[iteration_noise[2]]["demand"][n])
                }

fpi

{(1, 0): {'node': (1, 1),
  'var_out': {(1, 'i_coal'): 0.0,
   (2, 'i_coal'): 0.0,
   (3, 'i_coal'): 0.0,
   (1, 'x_line'): 0.0,
   (2, 'x_line'): 0.0,
   (3, 'x_line'): 0.0},
  'obj_val': 14500.0,
  'cos2go': 0.0,
  'stg_obj': 14500.0,
  'noise': (2, 400)},
 (1, 1): {'node': (3, 3),
  'var_out': {(1, 'i_coal'): 0.0,
   (2, 'i_coal'): 0.0,
   (3, 'i_coal'): 0.0,
   (1, 'x_line'): 0.0,
   (2, 'x_line'): 0.0,
   (3, 'x_line'): 0.0},
  'obj_val': 13500.0,
  'cos2go': 0.0,
  'stg_obj': 13500.0,
  'noise': (3, 500)}}

In [ ]:
        print("\nnode", node)
    
        if node == (1, 1):

            sb[node].x_line.domain = Binary        # This is because in the backward pass we relax these into continuous variables
            sb[node].x_line_in.domain = Binary
            sb[node].x_line_loc.domain = Binary


            for v in sb[1,1].component_objects(Param,active=True):
                 if hasattr(v, 'dummy') == True:
                    for index in v: 
                        if index == None:
                            setattr(sb[1,1], str(v), init_state[str(v)]) 
                        else:
                            setattr(sb[1,1], str(v[index]), init_state[str(v)][index]) 

            for n in OmegaBus:
                sb[1,1].demand[n].value = intra[iteration_noise[0]]["demand"][n]
        
        else:
            sb[node].x_line.domain = Binary        # This is because in the backward pass we relax these into continuous variables
            sb[node].x_line_in.domain = Binary
            sb[node].x_line_loc.domain = Binary


            for v in sb[node].component_objects(Param,active=True):
                 if hasattr(v, 'dummy') == True:
                    for index in v: 
                        if index == None:
                            setattr(sb[node], str(v), fpi[iter,step-1]['var_out'][(0,str(v)[:-2])]) 
                        else:
                            setattr(sb[node], str(v[index]), fpi[iter,step-1]['var_out'][(index, str(v)[:-2])]) 

            for n in OmegaBus:
                    sb[node].demand[n].value = intra[iteration_noise[step]]["demand"][n]
                            
        results = opt.solve(sb[node])
    
        out_dict = {}
        
        for v in sb[node].component_objects(Var,active=True):
             if hasattr(v, 'state_variable') == True:
                for index in v: 
                    if index == None:
                        out_dict[0, v.name] = value(v[index])
                    else:
                        out_dict[index, v.name] = value(v[index])
    
        fpi[iter,step] = {'node'  : node,
                     'var_out': out_dict,
                     'obj_val': value(sb[node].obj),
                     'cos2go': sb[node].cost_to_go.value,
                     'stg_obj':  value(sb[node].obj) - sb[node].cost_to_go.value,
                     'noise': "None"
                     }
    
        print("Cost to Go (Bellman Term):", fpi[iter,step]['cos2go'])
        print("Objective Value:",fpi[iter,step]['obj_val'])
        print("Stage Objective:", fpi[iter,step]['stg_obj'])


#### -----------------------------##### -------------------------------- ##### 

    print("\nbackward pass")
    
    β = {}
    α = {}
    
    V = {}
    dxdV = {}
    
    # -----------------stage 4 cuts -------------#

    for stage in reversed(Stages.stages):
        print("current stage", stage)

        if stage > 1 :
            
            for node in Stages.nodes[stage]:
                print("Current node", node)

                for v in sb[node].component_objects(Param,active=True):
                     if hasattr(v, 'dummy') == True:
                        for index in v: 
                            if index == None:
                                v.value = fpi[iter,stage-2]['var_out'][(0,str(v.name)[:-2])]
                                print("input of {} set to {}".format(v, v.value))
                            else:
                                v[index].value = fpi[iter,stage-2]['var_out'][(index, str(v.name)[:-2])]
                                print("input of {} is {}".format(v[index], v[index].value))




                if stage == Stages.stages[-1]:
                    sb[node].cost_to_go.fix(0)

                sb[node].x_line.domain = NonNegativeReals         ### Just a temporary form of relaxation because the problem needs to be linear to find the duals variables. 
                sb[node].x_line_in.domain = NonNegativeReals
                sb[node].x_line_loc.domain = NonNegativeReals

                for o in intra:
                    for n in OmegaBus:
                        sb[node].demand[n].value = intra[o]["demand"][n]
                                                
                    results = opt.solve(sb[node])
                    V[node,o] = value(sb[node].obj)

                    for v in sb[node].component_objects(Constraint,active=True):
                        if hasattr(v, 'dummy_constraint') == True:
                            for index in v: 
                                    dxdV[node,str(v)[4:-1],index,o] = sb[node].dual[v[index]]
                                    print("dual for ", v, "is", sb[node].dual[v[index]])
                                    
                for i in Nodes[node].parents:

                    for v in sb[node].component_objects(Var,active=True):
                        if hasattr(v, 'state_variable') == True:
                            for index in v:
                                β[iter, i, node, str(v), index]  = Pij[i,node]*sum(intra[o]["rho"]*dxdV[node, str(v), index, o] for o in intra)


                    α[iter, i,node] = Pij[i,node]*sum(intra[o]["rho"]*V[node,o] for o in intra)\
                                 - sum(β[iter, i,node,v.name[:-2],index]*v[index].value\
                                           for v in sb[node].component_objects(Param,active=True)\
                                           if hasattr(v, 'dummy') == True\
                                           for index in v)


    
            for i in Stages.nodes[stage-1]:
                sb[i].cuts.add(expr = sb[i].cost_to_go >= sum(α[iter, i,j] for j in Stages.nodes[stage])\
                                       + sum(sum(β[iter, i,j,v.name,index]*v[index] for j in Stages.nodes[stage])\
                                           for v in sb[i].component_objects(Var,active=True)\
                                           if hasattr(v, 'state_variable') == True for index in v))
        
    for o in intra:
        for n in OmegaBus:
            sb[1,1].demand[n].value = intra[o]["demand"][n]

        results = opt.solve(sb[1,1])
        V[(1,1),o] = value(sb[1,1].obj)
    lower_bound = sum(intra[o]["rho"]*V[(1,1),o] for o in intra)
    print(lower_bound)

In [232]:

# Define the conversion function
def pyomo_to_sympy(expr):
    """
    Convert a Pyomo expression to a SymPy expression, handling inequalities, arithmetic operations, 
    and specific Pyomo expression types including MonomialTermExpression.
    """
    # Check for Python numeric types first
    if isinstance(expr, (int, float)):
        return sp.Float(expr)
    elif expr.is_constant():
        # Convert Pyomo constants
        return sp.Float(value(expr))
    elif expr.is_variable_type():
        # Convert Pyomo variables to SymPy symbols
        return sp.Symbol(expr.name)
    elif expr.__class__.__name__ == 'InequalityExpression':
        # Handle inequality expressions (<=, >=)
        lhs = pyomo_to_sympy(expr.args[0])  # Left-hand side
        rhs = pyomo_to_sympy(expr.args[1])  # Right-hand side
        return lhs <= rhs  # Assuming it's a <= inequality
    elif expr.__class__.__name__ == 'SumExpression':
        # Handle sum expressions
        return sp.Add(*[pyomo_to_sympy(arg) for arg in expr.args])
    elif expr.__class__.__name__ == 'ProductExpression':
        # Handle product expressions
        return sp.Mul(*[pyomo_to_sympy(arg) for arg in expr.args])
    elif expr.__class__.__name__ == 'DivisionExpression':
        # Handle division expressions
        return pyomo_to_sympy(expr.args[0]) / pyomo_to_sympy(expr.args[1])
    elif expr.__class__.__name__ == 'PowerExpression':
        # Handle power expressions
        return sp.Pow(pyomo_to_sympy(expr.args[0]), pyomo_to_sympy(expr.args[1]))
    elif expr.__class__.__name__ == 'NegationExpression':
        # Handle negation expressions
        return -pyomo_to_sympy(expr.args[0])
    elif expr.__class__.__name__ == 'MonomialTermExpression':
        # Handle monomial terms (e.g., c * x)
        return sp.Mul(*[pyomo_to_sympy(arg) for arg in expr.args])
    else:
        raise ValueError(f"Unsupported expression type: {expr.__class__.__name__}")


# Convert the Pyomo expression to a SymPy expression


In [233]:
s = 2
n = 3

for i in range(1,3):
    sympy_expr = pyomo_to_sympy(sb[s,n].cuts[i].expr)
    # print("SymPy Expression:", sympy_expr)
    simplified_expr = sp.simplify(sympy_expr)
    print(simplified_expr)

1.0*cost_to_go + 2399.520024*i_coal[1] + 6398.720064*i_coal[2] + 2399.520024*i_coal[3] + 6398.720064*i_coal[4] + 6398.720064*i_coal[5] + 5670.53239005*x_line[10] + 3999.20004*x_line[11] >= 10902.81910905
1.0*cost_to_go + 2399.520024*i_coal[1] + 6398.720064*i_coal[2] + 2399.520024*i_coal[3] + 6398.720064*i_coal[4] + 6398.720064*i_coal[5] + 5670.53239005*x_line[10] + 3999.20004*x_line[11] >= 10902.81910905


In [234]:
sb[1,1].pprint()

64 Set Declarations
    con_i_coalada_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    con_i_coalx_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    con_x_lineada_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   15 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
    con_x_linex_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   15 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
    cuts_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    demand_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                        : Si